<h1 style="color:Blue;">CIFAR-10 image classification with TensorFlow<br>
    Part 2 - Preparing for deployment</h1>

<b>In Part 2 of our tutorial, we will use the Xilinx&trade; DNNDK toolsuite to convert our trained floating-point model into instructions and data for the DPU.
<br><br>
There are 4 main steps:
    <ul>
      <li>Converting variables to constants and stripping out training nodes (often referred to as 'freezing the graph')
      <li>Download images for quantization
      <li>Quantize the floating-point model using DECENT_Q
      <li>Compile the quantized model using DNNC          
    </UL>
<br>
We will not be using Pruning in this tutorial.
</b>

<h2 style="color:Blue;">Freeze the floating-point model</h2>

<b>Now that we have saved the trained parameters of our network as a checkpoint and graph, we need to 'freeze' it by converting all the variables into constants and stripping out the training nodes to leave just the nodes that we need for deployment.
<br><br>
Luckily, TensorFlow provides a script called 'freeze_graph.py' which will do this for us..</b>

In [ ]:
# remove previous results
!rm -rf ./freeze
!mkdir ./freeze

# freeze the inference graph
!freeze_graph --input_graph=./chkpts/inference_graph.pb \
              --input_checkpoint=./chkpts/float_model.ckpt \
              --input_binary=true \
              --output_graph=./freeze/frozen_graph.pb \
              --output_node_names=dense_1/BiasAdd

<h2 style="color:Blue;">Quantize the floating-point model</h2>

<b>This is the first step that involves the Xilinx&trade; DNNDK toolkit. We will now quantize the floating point model by running DECENT_Q to generate an 8bit model as required by the DPU.
</b>

<b>The quantization process has a calibration phase which requires approximately 1000 images files in a format that is compatible with openCV.
<br>
This section of python code will fetch the CIFAR-10 dataset as numpy arrays and the convert the first 1k arrays of the test dataset into 1k .png image files and write them into a separate folder.
<br>
It will also create a text file in that same folder which lists the images - this is also required for calibration.
</b>

In [ ]:
import os
import shutil
import numpy as np

from keras.preprocessing.image import save_img, array_to_img
from keras.datasets import cifar10

SCRIPT_DIR = os.getcwd()
CALIB_DIR = os.path.join(SCRIPT_DIR, 'calib_dir')
IMAGE_LIST_FILE = 'calib_list.txt'

# remove previous results
if (os.path.exists(CALIB_DIR)):
    shutil.rmtree(CALIB_DIR)
os.makedirs(CALIB_DIR)
print('Directory', CALIB_DIR, 'created') 

# fetch the CIFAR-10 dataset as numpy arrays
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# create file for list of calibration images
f = open(os.path.join(CALIB_DIR, IMAGE_LIST_FILE), 'w')

# use Keras array_to_img function to create .png files
for i in range(1000):
    img = array_to_img(x_test[i])
    save_img(os.path.join(CALIB_DIR,'x_test_'+str(i)+'.png'), img)
    f.write('x_test_'+str(i)+'.png\n')

f.close()


<b>Now we run the decent_q quantizer tool. This example calls decent_q without the use of a custom image processing function. The .png images we saved in the previous step are already of the correct shape and do not need to zoomed.
<br>
We do however apply a scaling factor to the pixel values using the `--scales` option with values of `0.00392,0.00392,0.00392` which corresponds to a division by 255.0 of each color channel, exactly as done during training.
<br>
Users are encouraged to refer to the DECENT_Q User Guide to see the complete list of command options.
</b>

In [ ]:
# delete previous results
!rm -rf ./quantize_results

!decent_q quantize \
  --input_frozen_graph ./freeze/frozen_graph.pb \
  --input_nodes images_in \
  --input_shapes ?,32,32,3 \
  --output_nodes dense_1/BiasAdd \
  --method 1 \
  --input_fn default \
  --calib_iter 100 \
  --batch_size 10 \
  --image_dir ./calib_dir \
  --image_list ./calib_dir/calib_list.txt \
  --scales 0.00392,0.00392,0.00392 \
  --gpu 0

<b>The quantize_results folder will contain two .pb files - quantize_eval_model.pb which we use to verify the quality of our now quantized model and deploy_model.pb which is the .pb file that will be passed to the DNNC compiler.</b>

<h2 style="color:Blue;">Evaluate the quantized model</h2>

<b>This is an optional step that can be skipped but is very useful in understanding how much the quantization process has effected the accuracy of our classifier.
</b>

<b>The evaluation process uses the quantized evaluation model (quantize_eval_model.pb) which was generated during the quantization process.
<br>
The top-1 and top-5 accuracies are calculated - the top-1 accuracy should be compared to the final accuracy calculated during training and evaluation to understand how much accuracy was lost to quantization.
</b>

In [ ]:
!python eval_graph.py \
  --graph ./quantize_results/quantize_eval_model.pb \
  --input_node images_in \
  --output_node dense_1/BiasAdd \
  --gpu 0

<h2 style="color:Blue;">Compile the quantized model</h2>

<b>
The final step is to use the DNNC compiler which will parse the quantized model and generate one or more .ELF format files which are to be integrated into the final hardware/software platform.
</b>

In [ ]:
# delete previous results
!rm -rf ./compile

!dnnc \
       --parser=tensorflow \
       --frozen_pb=./quantize_results/deploy_model.pb \
       --dpu=4096FA \
       --cpu_arch=arm64 \
       --output_dir=compile \
       --save_kernel \
       --mode normal \
       --net_name=cifar10

<b>The 'compile' folder should contain the .elf file for execution on the DPU.</b>